In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-27"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1251,2024-06-27,Venezuela Superliga,20:30,Gaiteros del Zulia,Diablos,1.10,6.14,155.5,1.82,1.84,0.0,0.00,0.00,SAxR7luH,0.909091,0.162866,0.549451,0.543478,0.071957,160.678,57.213908,0.356078,1.8,1.643168,0.912871,123.19,2.340,1.305929,0.558089,15.0,267.426,74.612482,0.279002,0.0,0.000000,NaN,365.31,3.498,0.902508,0.258007,-47.0,97,81,1.27,4.51,129.138,276.686,0.984480,0.007728,NaN,1.81,0.362,0.276243,0.000000,0.0,0.000000,-5.00,-1.000,-5.140000,0.000000,0.0,0.000000
1252,2024-06-27,Canadá Cebl,20:30,Ottawa Blackjacks,Scarborough Shooting Stars,2.21,1.56,181.5,1.80,1.86,0.0,0.00,0.00,ri8g8yEI,0.452489,0.641026,0.555556,0.537634,0.093514,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,91,105,2.37,1.45,0.000,0.000,0.243830,0.023184,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1253,2024-06-27,Canadá Cebl,21:00,Winnipeg Sea Bears,Niagara River,2.06,1.65,181.5,1.83,1.83,0.0,0.00,0.00,fTzNj9ro,0.485437,0.606061,0.546448,0.546448,0.091497,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,88,97,2.05,1.34,0.000,0.000,0.156288,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1254,2024-06-27,Colômbia Lbp,20:00,Cimarrones del Choco,Piratas de Bogota,1.50,2.47,166.5,1.80,1.86,0.0,0.00,0.00,UP52LttC,0.666667,0.404858,0.555556,0.537634,0.071525,241.022,148.955844,0.618018,1.8,1.643168,0.912871,160.72,2.948,1.960579,0.665054,-8.0,268.408,52.105521,0.194128,0.6,1.341641,2.236068,195.70,2.978,0.687183,0.230753,-36.0,98,95,1.64,2.06,202.508,219.962,0.345538,0.023184,NaN,-0.43,-0.086,-5.813953,0.000000,0.0,0.000000,-2.53,-0.506,-2.905138,0.000000,0.0,0.000000
1255,2024-06-27,México Cibacopa,23:00,Rayos de Hermosillo,Astros,1.98,1.70,181.5,1.84,1.82,1.5,1.85,1.61,z7hHa3ll,0.505051,0.588235,0.543478,0.549451,0.093286,187.512,40.519044,0.216088,2.4,1.341641,0.559017,163.20,2.154,0.598648,0.277924,35.0,147.536,23.687859,0.160556,1.2,1.643168,1.369306,186.12,1.546,0.262831,0.170007,-17.0,96,94,1.70,1.98,160.136,151.124,0.107603,0.007728,0.098096,1.89,0.378,2.592593,0.662646,0.7,0.037354,-2.09,-0.418,-1.674641,0.687526,0.6,-0.087526
1256,2024-06-27,Europa Campeonato Das Nações Europeias Menores,14:30,Andorra,Malta,3.61,1.26,144.5,1.84,1.86,0.0,0.00,0.00,Sf2ar9Er,0.277008,0.793651,0.543478,0.537634,0.070659,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,80,92,1.29,1.10,0.000,0.000,0.682423,0.007644,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1257,2024-06-27,Venezuela Superliga,20:00,Trotamundos,Llaneros,1.30,3.31,161.5,1.84,1.86,0.0,0.00,0.00,WrqI9AA4,0.769231,0.302115,0.543478,0.537634,0.071346,139.116,52.853161,0.379922,1.2,1.643168,1.369306,106.02,1.624,0.420809,0.259119,2.0,181.148,48.481217,0.267633,1.2,1.643168,1.369306,237.36,2.456,0.696836,0.283728,-13.0,93,69,1.14,3.44,129.884,251.988,0.616609,0.007644,NaN,0.97,0.194,1.546392,0.000000,0.0,0.000000,-5.00,-1.000,-2.310000,0.000000,0.0,0.000000
1258,2024-06-27,Mundo 

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
1255,23:00,México Cibacopa,Rayos de Hermosillo,Astros,1.98,Back Home
1273,21:00,Porto Rico Bsn,Capitanes De Arecibo,Mets de Guaynabo,1.76,Back Home
